In [4]:
import os
import pandas as pd
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation

In [5]:
api = tradeapi.REST(
    'AKA76KKPBG3JT8QG68VA',
    'HON2dhuWv3PajTXsx5IVBcVyFjlroG6Uq6YRrcxq',
    api_version = "v2"
)

In [6]:
timeframe ='1D'

In [7]:
start_date = pd.Timestamp("2010-01-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2020-10-29", tz="America/New_York").isoformat()


In [8]:
tickers = ["AAPL"]

In [9]:
df_ticker = api.get_barset(
    tickers,
    timeframe,
    start=start_date,
    end=end_date
).df

In [10]:
df_ticker.head()

AAPL                                   
                             open    high     low   close     volume
2010-01-04 00:00:00-05:00  30.487  30.643  30.340  30.597  116694866
2010-01-05 00:00:00-05:00  30.640  30.799  30.464  30.626  136014674
2010-01-06 00:00:00-05:00  30.626  30.747  30.107  30.134  133300804
2010-01-07 00:00:00-05:00  30.250  30.286  29.864  30.083  113809115
2010-01-08 00:00:00-05:00  30.043  30.286  29.866  30.270  104221988

In [11]:
MC_even_dist = MCSimulation(
    portfolio_data = df_ticker,
    weights = [1],
    num_simulation = 1000,
    num_trading_days = 252*5
)

In [12]:
MC_even_dist.calc_cumulative_return()

Running Monte Carlo simulation number 0.
Running Monte Carlo simulation number 10.
Running Monte Carlo simulation number 20.
Running Monte Carlo simulation number 30.
Running Monte Carlo simulation number 40.
Running Monte Carlo simulation number 50.
Running Monte Carlo simulation number 60.
Running Monte Carlo simulation number 70.
Running Monte Carlo simulation number 80.
Running Monte Carlo simulation number 90.
Running Monte Carlo simulation number 100.
Running Monte Carlo simulation number 110.
Running Monte Carlo simulation number 120.
Running Monte Carlo simulation number 130.
Running Monte Carlo simulation number 140.
Running Monte Carlo simulation number 150.
Running Monte Carlo simulation number 160.
Running Monte Carlo simulation number 170.
Running Monte Carlo simulation number 180.
Running Monte Carlo simulation number 190.
Running Monte Carlo simulation number 200.
Running Monte Carlo simulation number 210.
Running Monte Carlo simulation number 220.
Running Monte Carlo si

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
1,9.918131e-01,2.043157e+00,6.873916e-01,5.957065e-01,-9.443101e-01,2.356992e-01,1.542728e+00,9.985746e-01,1.015319e+00,1.729978e+00,...,5.277632e-01,-6.380955e-01,3.650291e+00,1.303088e+00,-5.430008e-01,2.279523e+00,1.972230e+00,-8.190146e-01,1.117893e+00,2.194688e+00
2,-1.693328e-01,1.449071e+00,6.414345e-01,1.858528e+00,-1.030568e+00,2.516846e-01,1.401249e+00,9.878508e-01,3.633549e+00,1.567022e+00,...,-2.204513e-01,-3.922011e-01,2.341995e+00,4.712257e+00,-1.483796e+00,-6.348853e-01,2.273077e-01,-1.696388e+00,2.297051e+00,-2.344024e-01
3,4.827077e-02,3.037164e+00,3.002486e-01,-1.074025e+00,-1.356982e+00,6.669485e-01,4.478723e+00,5.447685e-01,7.032507e+00,1.736479e+00,...,-3.504940e-01,-1.124927e+00,-8.662955e-01,-5.294085e-01,-3.414049e+00,-6.285995e-01,2.386591e-01,7.750228e-01,8.564393e+00,-4.025024e-01
4,1.173142e-01,2.442857e+00,-3.373732e-01,-2.343407e+00,-2.833788e+00,2.579155e+00,1.054324e+01,-3.130482e-01,5.191479e+00,4.836886e+00,...,-1.880713e-01,3.405123e-01,-1.171111e+00,-1.844816e+00,-2.734713e+00,-2.470376e-01,4.744670e-01,2.170657e+00,9.744259e+00,-4.227223e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1256,1.477673e+35,-1.182949e+26,-5.657196e+38,-4.856821e+56,3.716444e+40,8.331160e+41,-1.975680e+55,-5.257081e+34,-3.260585e+48,2.178997e+37,...,-1.794839e+39,-9.412809e+53,-4.850724e+27,2.353959e+13,1.042829e+45,2.255108e+41,-2.340628e+40,1.214392e+47,-2.509030e+51,-3.959449e+24
1257,2.196267e+35,-1.224142e+26,-1.004219e+39,5.232748e+56,5.736499e+40,2.905374e+41,6.171717e+54,9.743370e+33,-7.154144e+48,8.107784e+36,...,-7.665684e+39,-3.736138e+53,-1.174409e+28,9.398442e+12,9.212418e+44,2.941840e+41,-5.917743e+40,7.228566e+46,-6.909770e+51,3.119994e+24
1258,2.204668e+35,2.182311e+25,-1.938077e+39,1.834965e+57,-7.985183e+39,6.815035e+41,1.400133e+55,1.407341e+34,-6.934134e+48,-1.220836e+36,...,-2.224641e+40,-1.812585e+53,-2.300185e+28,2.042171e+13,6.950481e+44,7.546057e+41,-1.523939e+41,1.869976e+47,-1.628921e+52,9.819529e+23
1259,-1.248107e+35,-5.864224e+23,3.936911e+39,-9.470469e+56,-1.972844e+40,3.162519e+41,-5.235442e+54,-5.808809e+33,-2.841722e+48,-1.663591e+35,...,-4.492385e+40,-1.649166e+53,-4.384039e+28,9.648209e+11,2.839877e+44,2.754255e+42,-2.125390e+41,3.105028e+47,-7.088450e+52,1.340269e+24


In [13]:
even_tbl = MC_even_dist.summarize_cumulative_return()
print(even_tbl)

count           1.000000e+03
mean            3.067589e+81
std             9.700519e+82
min            -2.177083e+77
25%            -7.024997e+34
50%             4.899982e+04
75%             4.134356e+35
max             3.067573e+84
95% CI Lower   -8.198576e+59
95% CI Upper    5.084218e+61
Name: 1260, dtype: float64
